In [2]:
import torch
import torch.nn.functional as F 
import matplotlib as plt 
%matplotlib inline


In [36]:
words=open("names.txt",'r').read().splitlines()
chars=sorted(list(set(''.join(words))))
stoi={s:i+1 for i,s in enumerate(chars)}
stoi['.']=0
itos={i:s for s,i in stoi.items()}
vocab_size=len(itos)
vocab_size

27

27